# First Year Project - Project 1 - Corona and Weather

## Here importing the libraries required to run all the code
* [Pandas Documentation](https://pandas.pydata.org/docs/)
* [Numpy Documentation](https://numpy.org/doc/)
* [Folium Documentation](https://python-visualization.github.io/folium/)
* [Json Documentation](https://docs.python.org/3/library/json.html)
* [Statsmodels Documentation](https://www.statsmodels.org/stable/)
* [Scipy Documentation](https://scipy.github.io/devdocs/index.html)
* [Seaborn Documentation](https://seaborn.pydata.org/)

In [39]:
#Importing needed libraries
import pandas as pd
import numpy as np
import folium, json
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
from statsmodels.stats.multitest import multipletests
import seaborn as sns

### Task 0: Data filtering and cleaning

In [48]:
# Read comma-separated values (csv) file into pandas DataFrame
data = pd.read_csv("../../Data/raw/weather/weather.csv")

# A processed data set that contains only relevant fields and records 
is_DE = data[data['iso3166-2'].str.startswith('DE')]


is_DE.describe()

,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
count,6000.000000,6.000000e+03,6.000000e+03,6000.000000,6000.000000,6000.000000,6000.000000
mean,74.332120,6.678336e+06,2.379588e+06,283.281258,0.002206,16.050800,3.503221
std,13.595421,6.212940e+06,5.121144e+04,7.146610,0.003439,14.515517,1.438837
min,33.880265,0.000000e+00,2.212828e+06,260.531714,0.000000,0.000000,1.091346
25%,64.268213,1.090176e+06,2.343795e+06,277.837390,0.000054,2.500231,2.427226
50%,76.469758,4.610296e+06,2.385826e+06,283.228645,0.000684,12.720154,3.202055
75%,85.703146,1.150929e+07,2.422283e+06,288.872994,0.002907,27.398392,4.261427
max,98.264247,2.370823e+07,2.497243e+06,300.960922,0.031971,52.792235,11.221876


In [53]:
df = pd.read_csv("../../Data/processed/de_weather+cases.csv")
corona_df = pd.read_csv("../../Data/processed/de_corona.csv")
# case_by_region
# case_by_region = case_by_region.set_index(['iso3166-2', case_by_region.index])
# case_by_region
df.describe()


,confirmed_addition,deceased_addition,population,cases_pc,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
count,5580.000000,5580.000000,5.580000e+03,5.580000e+03,5580.000000,5.580000e+03,5.580000e+03,5580.000000,5580.000000,5580.000000,5580.000000
mean,428.464158,12.168638,5.303129e+06,7.997853e-05,74.197126,6.825184e+06,2.379476e+06,10.244079,0.002102,16.347402,3.390559
std,832.302604,25.863549,4.831030e+06,1.151281e-04,13.869995,6.267350e+06,5.163197e+04,7.249725,0.003390,14.559572,1.336541
min,1.000000,0.000000,6.829860e+05,5.576420e-08,33.880265,0.000000e+00,2.212828e+06,-12.618286,0.000000,0.000000,1.091346
25%,18.000000,0.000000,2.143145e+06,6.039941e-06,63.735166,1.083668e+06,2.343415e+06,4.681973,0.000041,2.094053,2.391155
50%,95.500000,1.000000,3.644826e+06,2.387780e-05,76.481065,4.919209e+06,2.385866e+06,10.330538,0.000605,14.510766,3.114391
75%,396.000000,10.000000,7.982448e+06,1.178654e-04,85.949911,1.178740e+07,2.422864e+06,15.877810,0.002718,27.699274,4.116958
max,6809.000000,213.000000,1.793265e+07,9.257132e-04,98.264247,2.370823e+07,2.497243e+06,27.810922,0.031971,52.792235,11.221876
